# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [ ]:
# Passo 1: Pegar Cotação de dólar/euro/ouro
# Passo 2: Importar base de dados e atualizar base
# Passo 3: Exportar base de dados atualizada

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()

#Dólar
navegador.get("https://www.google.com/")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação do dólar")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#Euro
navegador.get("https://www.google.com/")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação do euro")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace("," , ".")


navegador.quit()


In [3]:
print(cotacao_dolar)
print(cotacao_euro)
print(cotacao_ouro)

5.191
5.637
319.24


### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [2]:
import pandas as pd

tab = pd.read_excel("Produtos.xlsx")
display(tab)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [16]:
tab.loc[tab["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tab.loc[tab["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tab.loc[tab["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

tab["Preço de Compra"] = tab["Cotação"] * tab["Preço Original"]
tab["Preço de Venda"] = tab["Preço de Compra"] * tab["Margem"]

display(tab)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.0832,5083.149168,1.40,7116.408835
1,Carro Renault,4500.00,Euro,5.5154,24819.300000,2.00,49638.600000
2,Notebook Dell,899.99,Dólar,5.0832,4574.829168,1.70,7777.209586
3,IPhone,799.00,Dólar,5.0832,4061.476800,1.70,6904.510560
4,Carro Fiat,3000.00,Euro,5.5154,16546.200000,1.90,31437.780000
5,Celular Xiaomi,480.48,Dólar,5.0832,2442.375936,2.00,4884.751872
6,Joia 20g,20.00,Ouro,310.7700,6215.400000,1.15,7147.710000


### Agora vamos exportar a nova base de preços atualizada

In [17]:
tab.to_excel("Produtos Novos.xlsx", index = False)